In [1]:
from openai import OpenAI
import pandas as pd
import numpy as np
import json
import os
import re
import time
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate #where we change the AI "personality"
from dotenv import load_dotenv, find_dotenv

In [2]:
load_dotenv()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [3]:
_ = load_dotenv(find_dotenv()) # read local .env file

#openai.api_key = os.getenv("api.txt")
COMPLETIONS_MODEL = "text-davinci-002"
BETTER_COMPLETIONS_MODEL = "text-davinci-003" #for my purposes, this is better
LONG_MODEL = "gpt-3.5-turbo-16k"
REGULAR_MODEL = "gpt-3.5-turbo"
GPT_4 = "gpt-4-1106-preview"

In [4]:
os.chdir('/Users/chrissoria/Documents/Research/Categorization_AI_experiments')
current_directory = os.getcwd()
print(current_directory)

/Users/chrissoria/Documents/Research/Categorization_AI_experiments


In [5]:
survey_participant_input = "a19i" #enter column name here

UCNets = pd.read_excel("/Users/chrissoria/Documents/Research/UCNets_Classification/data/Raw_Cond_for_Coding_all_waves.xlsx", engine='openpyxl',sheet_name="JOINT_DATA",usecols=[survey_participant_input])
UCNets = UCNets[survey_participant_input].dropna().unique()  # Drop NaN values and get unique elements

survey_participant_responses = '; '.join(str(item) for item in UCNets) #what we will feed to the model

UCNets = pd.DataFrame(UCNets, columns=[survey_participant_input])
UCNets[survey_participant_input] = UCNets[survey_participant_input].astype(str).str.lower()
UCNets[survey_participant_input] = UCNets[survey_participant_input].str.strip()
UCNets = UCNets[UCNets[survey_participant_input] != ''].reset_index(drop=True) #trimming all empty rows

UCNets = UCNets.iloc[:150]

UCNets.head()

,a19i
0,relocated back to east coast - closer to my sons
1,move in together with my partner
2,"out of living with my friends, and into living..."
3,to take a new job in new york city (both becau...
4,wanted to live in my own place outside my pare...


Here, I'm trying to "force" the model to "think" in steps by first A. trying to process the response into its own words and B. having it interact with that object. That is, instead of all steps being given at once, I'm having it think in steps. 

This time, I will have it think in a "chain," where I will have it output a response and then feed that response back to it in a seperate prompt. 

In [6]:
def extract_categories(survey_question, 
                       survey_input,
                       user_model,
                       creativity,
                       *categories):
    
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    categories_str = "\n".join(f"{i + 1}. {cat}" for i, cat in enumerate(categories))
    cat_num = len(user_categories)
    category_dict = {str(i+1): "0" for i in range(cat_num)}
    example_JSON = json.dumps(category_dict, indent=4)
    
    link1 = []
    extracted_jsons = []
    
    for response in survey_input:
        prompt = f"""A survey respondent was asked, "{survey_question}" \
        Their response is here in triple backticks: ```{response}```. \
        First, select all of the following categories this response falls into: \
        "{categories_str}" \
        Second, form your response into JSON format like this example: {example_JSON}."""

        try:
            response = client.chat.completions.create(
                model=user_model,
                messages=[
                    {'role': 'user', 
                     'content': prompt}
                ],
                temperature=creativity
            )

            reply = response.choices[0].message.content
            link1.append(reply)
            
        except Exception as e:
            print(f"An error occurred: {e}")
            link1.append(f"Error processing input: {input}")
            
        extracted_json = re.findall(r'```json\n(\{.*?\})\n```', reply, re.DOTALL)
        extracted_json = extracted_json[0].replace('[', '').replace(']', '').replace('\n', '').replace(" ", '').replace("  ", '')
        print(extracted_json)
        extracted_jsons.append(extracted_json)
            
    normalized_data_list = []
    error_lines = []
    
    for i, json_str in enumerate(extracted_jsons):
        try:
            parsed_obj = json.loads(json_str)
            normalized_data_list.append(pd.json_normalize(parsed_obj))
        except json.JSONDecodeError:
            error_lines.append(i)
            continue

    normalized_data = pd.concat(normalized_data_list, ignore_index=True)
    
    categorized_data = pd.DataFrame()
    categorized_data['survey_response'] = survey_input
    categorized_data['link1'] = link1
    categorized_data['json'] = extracted_jsons
    
    categorized_data = pd.concat([categorized_data, normalized_data], axis=1)
    
    return categorized_data

In [7]:
survey_question = "Why did you move?"
survey_input = UCNets['a19i']

user_categories = ["to start living with or to stay with partner/spouse",
                   "relationship change (divorce, breakup, etc)",
                   "the person had a job or school or career change, including transferred and retired",
                   "the person's partner's job or school or career change, including transferred and retired",
                   "financial reasons (rent is too expensive, pay raise, etc)",
                   "related specifically features of the home, such as a bigger or smaller yard"]

user_model = 'gpt-4-1106-preview'

creativity = .25

First, I want to find out whether adjusting "creativity" makes a difference in how accurate the output is.

In [8]:
user_categories = ["to start living with or to stay with partner/spouse",
                   "relationship change (divorce, breakup, etc)",
                   "the person had a job or school or career change, including transferred and retired"]

cat_3 = extract_categories(survey_question, 
                            survey_input, 
                            user_model,
                            creativity,
                            user_categories)

cat_3.to_csv('data/a19i/a19i_3_cats_no_cot.csv', index=False)

{"1":"0","2":"0","3":"0"}
{"1":"1","2":"0","3":"0"}
{"1":"1","2":"0","3":"0"}
{"1":"0","2":"0","3":"1"}
{"1":"0","2":"0","3":"0"}
{"1":"0","2":"0","3":"0"}
{"1":"0","2":"0","3":"1"}
{"1":"0","2":"0","3":"0"}
{"1":"0","2":"0","3":"0"}
{"1":"1","2":"0","3":"0"}
{"1":"1","2":"0","3":"0"}
{"1":"0","2":"0","3":"1"}
{"1":"0","2":"0","3":"0"}
{"1":"0","2":"0","3":"0"}
{"1":"0","2":"0","3":"1"}
{"1":"0","2":"0","3":"0"}
{"1":"0","2":"0","3":"0"}
{"1":"0","2":"0","3":"0"}
{"1":"0","2":"0","3":"0"}
{"1":"1","2":"0","3":"0"}
{"1":"0"}
{"tostartlivingwithortostaywithpartner/spouse":"1","relationshipchange(divorce,breakup,etc)":"0","thepersonhadajoborschoolorcareerchange,includingtransferredandretired":"0"}
{"1":"0","2":"0","3":"0"}
{"1":"0","2":"0","3":"0"}
{"1":"0","2":"0","3":"0"}
{"1":"0","2":"0","3":"0"}
{"1":"0","2":"0","3":"0"}
{"1":"0","2":"0","3":"0"}
{"1":"0","2":"0","3":"0"}
{"1":"0","2":"0","3":"0"}
{"1":"0","2":"0","3":"0"}
{"1":"0"}
{"1":"0","2":"0","3":"1"}
{"1":"0","2":"0","3":"0"}


In [16]:
len(cat_3)

150

In [17]:
user_categories = ["to start living with or to stay with partner/spouse",
                   "relationship change (divorce, breakup, etc)",
                   "the person had a job or school or career change, including transferred and retired",
                   "the persons's partner's job or school or career change, including transferred and retired"]

cat_4 = extract_categories(survey_question, 
                            survey_input, 
                            user_model,
                            creativity,
                            user_categories)

cat_4.to_csv('data/a19i/a19i_4_cats_no_cot.csv', index=False)

print(len(cat_4))

{"1":"0","2":"0","3":"0","4":"0"}
{"1":"1","2":"0","3":"0","4":"0"}
{"1":"1","2":"0","3":"0","4":"0"}
{"1":"0","2":"0","3":"1","4":"0"}
{"1":"0","2":"0","3":"0","4":"0"}
{"1":"0","2":"0","3":"0","4":"0"}
{"1":"0","2":"0","3":"1","4":"0"}
{"1":"0","2":"0","3":"0","4":"0"}
{"1":"0","2":"0","3":"0","4":"0"}
{"1":"1","2":"0","3":"0","4":"0"}
{"1":"1","2":"0","3":"0","4":"0"}
{"1":"0","2":"0","3":"1","4":"0"}
{"1":"0","2":"0","3":"0","4":"0"}
{"1":"0","2":"0","3":"0","4":"0"}
{"1":"0","2":"0","3":"1","4":"0"}
{"1":"0","2":"0","3":"0","4":"0"}
{"1":"0","2":"0","3":"0","4":"0"}
{"1":"0","2":"0","3":"0","4":"0"}
{"1":"0","2":"0","3":"0","4":"0"}
{"1":"1","2":"0","3":"0","4":"0"}
{"1":"0","2":"0","3":"0","4":"0"}
{"1":"1","2":"0","3":"0","4":"0"}
{"1":"0","2":"0","3":"0","4":"0"}
{"1":"0","2":"0","3":"0","4":"0"}
{"1":"0","2":"0","3":"0","4":"0"}
{"1":"0","2":"0","3":"0","4":"0"}
{"1":"0","2":"0","3":"0","4":"0"}
{"1":"0","2":"0","3":"0","4":"0"}
{"1":"0","2":"0","3":"0","4":"0"}
{"1":"0","2":"

In [18]:
user_categories = ["to start living with or to stay with partner/spouse",
                   "relationship change (divorce, breakup, etc)",
                   "the person had a job or school or career change, including transferred and retired",
                   "the person's partner's job or school or career change, including transferred and retired",
                   "financial reasons (rent is too expensive, pay raise, etc)"]

cat_5 = extract_categories(survey_question, 
                            survey_input, 
                            user_model,
                            creativity,
                            user_categories)

cat_5.to_csv('data/a19i/a19i_5_cats_no_cot.csv', index=False)

{"1":"0","2":"0","3":"0","4":"0","5":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1"}
{"1":"0","2":"0","3":"1","4":"0","5":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"1"}
{"1":"0","2":"0","3":"0","4":"0","5":"1"}
{"1":"0","2":"0","3":"0","4":"0","5":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1"}
{"1":"0","2":"0","3":"0","4":"0","5":"1"}
{"1":"0","2":"0","3":"0","4":"0","5":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1"}
{"1":"0","2":"0","3":"0","4":"0","

In [20]:
print(len(cat_5))

150


In [21]:
user_categories = ["to start living with or to stay with partner/spouse",
                   "relationship change (divorce, breakup, etc)",
                   "the person had a job or school or career change, including transferred and retired",
                   "the person's partner's job or school or career change, including transferred and retired",
                   "financial reasons (rent is too expensive, pay raise, etc)",
                   "related specifically features of the home, such as a bigger or smaller yard"]

cat_6 = extract_categories(survey_question, 
                            survey_input, 
                            user_model,
                            creativity,
                            user_categories)

cat_6.to_csv('data/a19i/a19i_6_cats_no_cot.csv', index=False)
print(len(cat_6))

{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"1","6":"1"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"1"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0"}


In [22]:
user_categories = ["to start living with or to stay with partner/spouse",
                   "relationship change (divorce, breakup, etc)",
                   "the person had a job or school or career change, including transferred and retired",
                   "the person's partner's job or school or career change, including transferred and retired",
                   "financial reasons (rent is too expensive, pay raise, etc)",
                   "related specifically features of the home, such as a bigger or smaller yard",
                   "living environment unrelated to their specific home, such as a better or more exciting neighborhood."]

cat_7 = extract_categories(survey_question, 
                            survey_input, 
                            user_model,
                            creativity,
                            user_categories)

cat_7.to_csv('data/a19i/a19i_7_cats_no_cot.csv', index=False)
print(len(cat_7))

{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"1"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"1"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"1","6":"1","7":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"1","7":"0"}
{"1":"0","2":"

{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0"}
{"1":"0","2":"1","3":"0","4":"0","5":"1","6":"0","7":"1"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0"}
{"1":"0","2":"1","3":"0","4":"0","5":"0","6":"0","7":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0"}
{"1":"0","2":"1","3":"0","4":"0","5":"1","6":"0","7":"0"}
150


In [23]:
user_categories = ["to start living with or to stay with partner/spouse",
                   "relationship change (divorce, breakup, etc)",
                   "the person had a job or school or career change, including transferred and retired",
                   "the person's partner's job or school or career change, including transferred and retired",
                   "financial reasons (rent is too expensive, pay raise, etc)",
                   "related specifically features of the home, such as a bigger or smaller yard",
                   "living environment unrelated to their specific home, such as a better or more exciting neighborhood.",
                   "moved closer to family members for various purposes, such as providing support"]

cat_8 = extract_categories(survey_question, 
                            survey_input, 
                            user_model,
                            creativity,
                            user_categories)

cat_8.to_csv('data/a19i/a19i_8_cats_no_cot.csv', index=False)
print(len(cat_8))

{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"1"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"1","8":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"0","8":"1"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"1","8":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"1","6":"1","7":"0","8":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"0","8":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0"}
{"1":"0","

{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"1","7":"0","8":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"0","8":"1"}
{"1":"1","2":"0","3":"0","4":"1","5":"0","6":"0","7":"0","8":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"1"}
{"1":"0","2":"1","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"1","8":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"1","8":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"1","8":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"1"}
{"1":"0","

In [24]:
user_categories = ["to start living with or to stay with partner/spouse",
                   "relationship change (divorce, breakup, etc)",
                   "the person had a job or school or career change, including transferred and retired",
                   "the person's partner's job or school or career change, including transferred and retired",
                   "financial reasons (rent is too expensive, pay raise, etc)",
                   "related specifically features of the home, such as a bigger or smaller yard",
                   "living environment unrelated to their specific home, such as a better or more exciting neighborhood.",
                   "moved closer to family members for various purposes, such as providing support",
                   "to be closer to friends or expand their social circle"]

cat_9 = extract_categories(survey_question, 
                            survey_input, 
                            user_model,
                            creativity,
                            user_categories)

cat_9.to_csv('data/a19i/a19i_9_cats_no_cot.csv', index=False)
print(len(cat_9))

{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"1","9":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"1","8":"0","9":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"0","8":"1","9":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"1","8":"0","9":"1"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"1","6":"1","7":"0","8":"0","9":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"0","8":"0","9":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"

{"1":"0","2":"0","3":"1","4":"0","5":"1","6":"0","7":"0","8":"0","9":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"1","8":"0","9":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"1","7":"0","8":"0","9":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"1","7":"0","8":"0","9":"0"}
{"1":"0","2":"0","3":"0","4":"1","5":"0","6":"0","7":"0","8":"0","9":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"1","7":"0","8":"0","9":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"

In [25]:
user_categories = ["to start living with or to stay with partner/spouse",
                   "relationship change (divorce, breakup, etc)",
                   "the person had a job or school or career change, including transferred and retired",
                   "the person's partner's job or school or career change, including transferred and retired",
                   "financial reasons (rent is too expensive, pay raise, etc)",
                   "related specifically features of the home, such as a bigger or smaller yard",
                   "living environment unrelated to their specific home, such as a better or more exciting neighborhood.",
                   "moved closer to family members for various purposes, such as providing support",
                   "to be closer to friends or expand their social circle",
                   "conflicts with living mates, family members, landlord, or neighbors"]

cat_10 = extract_categories(survey_question, 
                            survey_input, 
                            user_model,
                            creativity,
                            user_categories)

cat_10.to_csv('data/a19i/a19i_10_cats_no_cot.csv', index=False)
print(len(cat_10))

{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"1","9":"0","10":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"1","8":"0","9":"0","10":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"0","8":"1","9":"0","10":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"1","8":"0","9":"1","10":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"1","6":"1","7":"0","8":"0","9":"0","10":"0"}
{"1"

{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"1","6":"0","7":"0","8":"1","9":"0","10":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"1","9":"0","10":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"0","8":"0","9":"0","10":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"0","8":"0","9":"0","10":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"1","9":"0","10":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"0","8":"0","9":"0","10":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"1","7":"0","8":"0","9":"0","10":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"1"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"0","8":"0","9":"0","10":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"1","6":"0","7":"0","8":"0","9":"0","10":"1"}
{"1"

In [26]:
user_categories = ["to start living with or to stay with partner/spouse",
                   "relationship change (divorce, breakup, etc)",
                   "the person had a job or school or career change, including transferred and retired",
                   "the person's partner's job or school or career change, including transferred and retired",
                   "financial reasons (rent is too expensive, pay raise, etc)",
                   "related specifically features of the home, such as a bigger or smaller yard",
                   "living environment unrelated to their specific home, such as a better or more exciting neighborhood.",
                   "moved closer to family members for various purposes, such as providing support",
                   "to be closer to friends or expand their social circle",
                   "conflicts with living mates, family members, landlord, or neighbors",
                   "compelled to move due to circumstances beyond their control, such as being evicted, lease cancellation"]

cat_11 = extract_categories(survey_question, 
                            survey_input, 
                            user_model,
                            creativity,
                            user_categories)

cat_11.to_csv('data/a19i/a19i_11_cats_no_cot.csv', index=False)
print(len(cat_11))

{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"1","9":"0","10":"0","11":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"1","8":"0","9":"0","10":"0","11":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"0","8":"1","9":"0","10":"0","11":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"1","8":"0","9":"1","10":"0","11":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0

{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"1","8":"0","9":"0","10":"0","11":"0"}
{"1":"0","2":"1","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"1","7":"0","8":"0","9":"0","10":"0","11":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"1","7":"0","8":"0","9":"0","10":"0","11":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"1"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"1","7":"0","8":"0","9":"0","10":"0","11":"1"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"1","6":"0","7":"0","8":"1","9":"0","10":"0","11":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"1","9":"1","10":"0

In [27]:
user_categories = ["to start living with or to stay with partner/spouse",
                   "relationship change (divorce, breakup, etc)",
                   "the person had a job or school or career change, including transferred and retired",
                   "the person's partner's job or school or career change, including transferred and retired",
                   "financial reasons (rent is too expensive, pay raise, etc)",
                   "related specifically features of the home, such as a bigger or smaller yard",
                   "living environment unrelated to their specific home, such as a better or more exciting neighborhood.",
                   "moved closer to family members for various purposes, such as providing support",
                   "to be closer to friends or expand their social circle",
                   "conflicts with living mates, family members, landlord, or neighbors",
                   "compelled to move due to circumstances beyond their control, such as being evicted, lease cancellation",
                   "a response that doesn't provide sufficient content or meaningful information or cannot be categorized for whatever reason"]

cat_12 = extract_categories(survey_question, 
                            survey_input, 
                            user_model,
                            creativity,
                            user_categories)

cat_12.to_csv('data/a19i/a19i_12_cats_no_cot.csv', index=False)
print(len(cat_12))

{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"1","9":"0","10":"0","11":"0","12":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"1","8":"0","9":"0","10":"0","11":"0","12":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"0","8":"1","9":"0","10":"0","11":"0","12":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"1","8":"0","9":"1","10":"0","11":"0","12":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"1"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0",

{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0"}
{"1":"0","2":"1","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0"}
{"1":"0","2":"1","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"1"}
{"1":"0","2":"1","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0"}
{"1":"0","2":"0","3":"0","4":"1","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"1","8":"0","9":"0","10":"0","11":"0",

In [28]:
user_categories = ["to start living with or to stay with partner/spouse",
                   "relationship change (divorce, breakup, etc)",
                   "the person had a job or school or career change, including transferred and retired",
                   "the person's partner's job or school or career change, including transferred and retired",
                   "financial reasons (rent is too expensive, pay raise, etc)",
                   "related specifically features of the home, such as a bigger or smaller yard",
                   "living environment unrelated to their specific home, such as a better or more exciting neighborhood.",
                   "moved closer to family members for various purposes, such as providing support",
                   "to be closer to friends or expand their social circle",
                   "conflicts with living mates, family members, landlord, or neighbors",
                   "compelled to move due to circumstances beyond their control, such as being evicted, lease cancellation",
                   "a response that doesn't provide sufficient content or meaningful information or cannot be categorized for whatever reason",
                   "a reason for moving that does not fit into any other category above"]

cat_13 = extract_categories(survey_question, 
                            survey_input, 
                            user_model,
                            creativity,
                            user_categories)

cat_13.to_csv('data/a19i/a19i_13_cats_no_cot.csv', index=False)
print(len(cat_13))

{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"1","9":"0","10":"0","11":"0","12":"0","13":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"1","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"1"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"0","8":"1","9":"0","10":"0","11":"0","12":"0","13":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"1","8":"0","9":"1","10":"0","11":"0","12":"0","13":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"1","13":"0"}
{"1":"1","

{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"1","9":"0","10":"0","11":"0","12":"0","13":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"1","8":"0","9":"1","10":"0","11":"0","12":"0","13":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"1","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"1","12":"0","13":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"1","9":"0","10":"0","11":"0","12":"0","13":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"1","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0"}
{"1":"0","

In [29]:
user_categories = ["to start living with or to stay with partner/spouse",
                   "relationship change (divorce, breakup, etc)",
                   "the person had a job or school or career change, including transferred and retired",
                   "the person's partner's job or school or career change, including transferred and retired",
                   "financial reasons (rent is too expensive, pay raise, etc)",
                   "related specifically features of the home, such as a bigger or smaller yard",
                   "living environment unrelated to their specific home, such as a better or more exciting neighborhood.",
                   "moved closer to family members for various purposes, such as providing support",
                   "to be closer to friends or expand their social circle",
                   "conflicts with living mates, family members, landlord, or neighbors",
                   "compelled to move due to circumstances beyond their control, such as being evicted, lease cancellation",
                   "a response that doesn't provide sufficient content or meaningful information or cannot be categorized for whatever reason",
                   "for health-related reasons",
                   "a reason for moving that does not fit into any other category above"]

cat_14 = extract_categories(survey_question, 
                            survey_input, 
                            user_model,
                            creativity,
                            user_categories)

cat_14.to_csv('data/a19i/a19i_14_cats_no_cot.csv', index=False)
print(len(cat_14))

{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"1","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"1","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"1"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"0","8":"1","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"1","8":"0","9":"1","10":"0","11":"0","12":"0","13":"0","14":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0"

{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"1","12":"0","13":"0","14":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"1","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"1"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"1","7":"1","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"1"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0"}
{"1":"0","2":"0","3":"0","4":"1","5":"1","6":"0","7":"0","8":"1","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"1","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"1","8":"0","9":"1","10":"0","11":"0","12":"0","13":"0","14":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0"

{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"1","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"0","8":"0","9":"0","10":"0","11":"1","12":"0","13":"0","14":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0"}
{"1":"0","2":"1","3":"0","4":"0","5":"1","6":"0","7":"1","8":"0","9":"1","10":"0","11":"0","12":"0","13":"0","14":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"1","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0"}
{"1":"0","2":"1","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0"

In [30]:
user_categories = ["to start living with or to stay with partner/spouse",
                   "relationship change (divorce, breakup, etc)",
                   "the person had a job or school or career change, including transferred and retired",
                   "the person's partner's job or school or career change, including transferred and retired",
                   "financial reasons (rent is too expensive, pay raise, etc)",
                   "related specifically features of the home, such as a bigger or smaller yard",
                   "living environment unrelated to their specific home, such as a better or more exciting neighborhood.",
                   "moved closer to family members for various purposes, such as providing support",
                   "to be closer to friends or expand their social circle",
                   "conflicts with living mates, family members, landlord, or neighbors",
                   "compelled to move due to circumstances beyond their control, such as being evicted, lease cancellation",
                   "a response that doesn't provide sufficient content or meaningful information or cannot be categorized for whatever reason",
                   "health-related",
                   "a natural disaster or concern about a potential natural disaster",
                   "a reason for moving that does not fit into any other category above"]

cat_15 = extract_categories(survey_question, 
                            survey_input, 
                            user_model,
                            creativity,
                            user_categories)

cat_15.to_csv('data/a19i/a19i_15_cats_no_cot.csv', index=False)
print(len(cat_15))

{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"1","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0","15":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0","15":"0"}
{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0","15":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"1","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0","15":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0","15":"1"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"0","8":"1","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0","15":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0","15":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"1","8":"0","9":"1","10":"0","11":"0","12":"0","13"

{"1":"1","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0","15":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"1","10":"0","11":"0","12":"0","13":"0","14":"0","15":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"1","11":"0","12":"0","13":"0","14":"0","15":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0","15":"1"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"1","12":"0","13":"0","14":"0","15":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"1","7":"0","8":"1","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0","15":"1"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"1","7":"1","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0","15":"1"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13"

{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"1","9":"0","10":"0","11":"1","12":"0","13":"0","14":"0","15":"1"}
{"1":"0","2":"1","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0","15":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"1","12":"0","13":"0","14":"0","15":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"1","12":"0","13":"0","14":"0","15":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"1","8":"0","9":"0","10":"1","11":"0","12":"0","13":"0","14":"0","15":"1"}
{"1":"0","2":"0","3":"0","4":"0","5":"1","6":"0","7":"0","8":"0","9":"0","10":"1","11":"0","12":"0","13":"0","14":"0","15":"0"}
{"1":"0","2":"0","3":"1","4":"0","5":"0","6":"0","7":"0","8":"0","9":"0","10":"0","11":"0","12":"0","13":"0","14":"0","15":"0"}
{"1":"0","2":"0","3":"0","4":"0","5":"0","6":"0","7":"1","8":"0","9":"0","10":"0","11":"0","12":"0","13"